# Problem Statement

When a field gets added to a live REDCap, CRPs would have to manually go through all the pre-existing records to fill out that field. 

Similarly, when a field currently has wrong value for multiple records and need its value changed, CRPs need to manually go in and fix these data points.

This manual process may introduce human errors and take too much time for the CRPs.

# Solution

This notebook lets the user specify the filter on the records and the fields that need new/updated values. 

Then, it exports only the records that need to be changed. Changes are made on this export before getting re-imported to the project.

**The notebook makes these changes in the Test project before actually making the changes in the Target project. Pay attention to the changes in the Test project and make sure of data integrity before moving forward with the Target project.**

# Instructions

### **TEST WITH TEST PROJECT BEFORE MAKING CHANGES TO THE TARGET PROJECT.**
### Instructions: 
1. Copy the Target project's data into the Test project's data use the notebook `copy_records_redcap.ipynb`.
3. Run this notebook.

# Code
### 1. Setup: Specify condition & fields, get project and project data

In [ ]:
import AMBRA_Backups
import json
from datetime import datetime
from Change.Change import Change

In [ ]:
# TEST PROJECT
test_project_name = "Personal Test"
test_project = AMBRA_Backups.redcap_funcs.get_redcap_project(test_project_name)

# TARGET PROJECT
target_project_name = "I3C DECADE - Image Tracking"
target_project = AMBRA_Backups.redcap_funcs.get_redcap_project(target_project_name)

In [ ]:
# FILTERS
filter_logic = """
   [patient_status_1] = ''
"""

# LIST OF CHANGES NEEDED
changes_needed = [{"field_name": "patient_status_1", "old_value": "", "new_value": "1"}]

changes_needed = [Change(field_name="patient_status_1", old_value="", new_value="1")]

In [ ]:
def update_fields(records: list, changes_needed: list):
    """
    Change the fields in `update_fields` into specified values in `records`.
    """

    for record in records:
        for change in changes_needed:
            field_name = change["field_name"]
            current_value = record[field_name]

            # Update value
            if current_value == change["old_value"]:
                record[field_name] = change["new_value"]

    return records

### 2. Testing with Test project

In [ ]:
# Write current records of Test project to Excel file for later comparison
test_records_df = test_project.export_records(
    format_type="df", export_blank_for_gray_form_status=True
)
test_records_df.to_excel(f"{test_project_name}_before_update_{datetime.now()}.xlsx")

In [ ]:
# Get records with filters from test project
test_filtered_records = test_project.export_records(
    filter_logic=filter_logic, export_blank_for_gray_form_status=True
)
test_filtered_records

In [ ]:
# Update the records
test_updated_records = update_fields(test_filtered_records, changes_needed)
test_updated_records

In [ ]:
# Reimport back to Test project
test_project.import_records(test_updated_records)

### Go to `compare_projects.ipynb` to compare the Test project's data before vs. after being updated.
After done comparing, set `compared = True`

In [ ]:
compared = True

if not compared:
    raise Exception("Read above instructions.")

### 3. Fill fields in Target project


In [ ]:
# Write current records of Target project to Excel file for later comparison
target_records_df = target_project.export_records(
    format_type="df", export_blank_for_gray_form_status=True
)
target_records_df.to_excel(f"{target_project_name}_before_update_{datetime.now()}.xlsx")

# Get JSON of current data in case anything goes wrong
all_records_json = target_project.export_records(export_blank_for_gray_form_status=True)
json_obj = json.dumps(all_records_json)

with open(f"{target_project_name}_{datetime.now()}.json", "w") as f:
    f.write(json_obj)

In [ ]:
# Get records with filters from Target project
target_filtered_records = target_project.export_records(
    filter_logic=filter_logic, export_blank_for_gray_form_status=True
)

# Update the records
target_updated_records = update_fields(target_filtered_records, changes_needed)

# Reimport back to Test project
target_project.import_records(target_updated_records)

### Go to `compare_projects.ipynb` to compare the Target project's data before vs. after being updated.
If anything goes wrong, you can always import the old data back.